# Using PhiData Agent to build an assistant for a process training


In this notebook, we will use **PhiData**, a framework for defining and running AI agents for specific tasks. This notebook will guide you through setting up an agent, assigning a task, and running the process.

## Steps Overview
1. Define a Knowledge Base (RAG) based on a document
2. Define an agent with LLM, Knowledge base reference.
3. Initate Agent in interactive mode

---
### Prerequisites
- Install the `phidata` library.
- Obtain API keys for the LLM model you want to use (e.g., Groq or OpenAI).
- Set up a Python environment with necessary dependencies.

---
### Code Walkthrough
Below is the implementation to define and use an AI agent for answering queries.

### Step 0 : Required installation and import of dependencies

In [1]:
!pip install pypdf phidata sentence-transformers groq lancedb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.9/716.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.6/32.6 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

#### imports

In [2]:
from phi.agent import Agent
from phi.model.groq import Groq
from phi.model.openai import OpenAIChat
from phi.storage.agent.json import JsonFileAgentStorage
from datetime import datetime
from phi.vectordb.lancedb import LanceDb
from phi.vectordb.search import SearchType
from phi.knowledge.pdf import PDFKnowledgeBase, PDFReader
from phi.document.chunking.fixed import FixedSizeChunking
from phi.embedder.sentence_transformer import SentenceTransformerEmbedder

### Step 1 : Load a PDF document into Knowledge Base

#### Mount your GDrive to access the file

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Create a vector DB using LanceDB (its simple, in-file DB)
#### Define a Knowledge base from a PDF document with chuking of its content
---

The PDF document of **ISO 27001** is loaded. This is a standard for requirements on information system security for organisations

In [4]:
# LanceDB Vector DB
vector_db = LanceDb(
    table_name="iso_27001",
    uri="Lance_KB/iso",
    search_type=SearchType.vector,
    embedder=SentenceTransformerEmbedder(model='all-MiniLM-L6-v2')
)

# Define knowledge base from a PDF document, using reader and chunking strategies provided by frame work
knowledge_base = PDFKnowledgeBase(
    path="/content/drive/My Drive/GrowthSchool_RAG_and_AgenticAI/Agentic_AI/Doc/iso_27001_2.pdf",
    vector_db=vector_db,
    reader=PDFReader(),
    chunking_strategy=FixedSizeChunking(chunk_size=300, overlap=30)
)

# Load the Knowledge base
knowledge_base.load(recreate=True)


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

INFO     Dropping collection

INFO     Creating collection

INFO     Loading knowledge base

INFO     Reading: iso_27001_2

INFO     Added 234 documents to knowledge base

### Step 2 : Define Agent with Knowledge base associated

Define an agent with LLM linked. Also the knowledge base is associated to it and parameters set to search for information in the knowledge base

In [5]:
from google.colab import userdata
import os

# Set your Groq API key or any other LLM API key
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
# os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

I_agent = Agent (
                # model=Groq (id="llama3-70b-8192"),
                # model=OpenAIChat (id="gpt-4o-mini"),
                model=Groq (id="llama-3.3-70b-versatile"),
                name = 'SME',
                agent_id = 'SME',
                description = "You are good at clarifying queries based on knowledge base",
                add_history_to_messages=True,
                role = 'Information Gatherer',
                storage=JsonFileAgentStorage("./tmp/agent_sessions_json"),
                instructions=["Your response shall be only based on Knowledge base",
                              "Search for specific information for query from knowledge base",
                              "Topics not related to knowledge base : Don't respond to original question",
                              "Provide your summary"],
                show_tool_calls=True,
                knowledge=knowledge_base,
                search_knowledge=True,
              )

### Step 3 : Launch agent in interactive mode

In [8]:
prompt = 'x'

while (prompt != 'exit'):
  prompt = input ("Enter your query ... or 'exit'")
  if prompt != 'exit':

    Res = I_agent.run (prompt)
    print (Res.content)

Enter your query ... or 'exit'scope of iso 9001?

Running:
 - search_knowledge_base(query=scope of iso 9001)


Running:
 - search_knowledge_base(query=ISO 9001 standard scope)

I cannot find relevant information in my knowledge base to answer this query.
Enter your query ... or 'exit'exit


In [7]:
from google.colab import userdata
import os

# Set your Groq API key or any other LLM API key
# os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

I_agent = Agent (
                # model=Groq (id="llama3-70b-8192"),
                model=OpenAIChat (id="gpt-4o-mini"),
                # model=Groq (id="llama-3.3-70b-versatile"),
                name = 'SME',
                agent_id = 'SME',
                description = "You are good at clarifying queries based on knowledge base",
                add_history_to_messages=True,
                role = 'Information Gatherer',
                storage=JsonFileAgentStorage("./tmp/agent_sessions_json"),
                system_prompt="""You are an assistant that answers questions **only** based on queried information from knowledge base.
                If no relevant information is found in the knowledge base, respond with:
                'I cannot find relevant information in my knowledge base to answer this query.'
                Do not answer from general knowledge.""",
                instructions=["Provide your summary"],
                show_tool_calls=True,
                knowledge=knowledge_base,
                retrieval_settings={
                    "top_k": 5,  # Number of information to retrieve
                    "score_threshold": 0.75  # Ensure only relevant information are considered
                },
                search_knowledge=True,
              )